# Setup

In [1]:
import os
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np

import seaborn as sns

import statsmodels.api as sm
from sklearn import linear_model

import statsmodels.formula.api as smf

In [2]:
# setting directories for data i/o

data_directory = 'C:/Users/charl/Documents/CNT 2020/Evictions/cnt-evictions/data/'
export_directory = 'C:/Users/charl/Documents/CNT 2020/Evictions/cnt-evictions/analysis/exports/'

analysis_data_file_name = 'joined_analysis_df_20200405_0000.csv'

In [19]:
# updating pandas view settings

pd.set_option('display.max_columns', 999)

export_files = True

## Import cleaned data analysis dataset

In [5]:
analysis_df = pd.read_csv(data_directory+analysis_data_file_name, low_memory=False)

In [6]:
analysis_df.head()

,Unnamed: 0,blkgrp,cbsa,blkgrps,population_x,households,land_acres,ht_ami,ht_80ami,ht_nmi,h_ami,h_80ami,h_nmi,t_ami,t_80ami,t_nmi,co2_per_hh_local,co2_per_acre_local,autos_per_hh_ami,autos_per_hh_80ami,autos_per_hh_nmi,vmt_per_hh_ami,vmt_per_hh_80ami,vmt_per_hh_nmi,pct_transit_commuters_ami,pct_transit_commuters_80ami,pct_transit_commuters_nmi,t_cost_ami,t_cost_80ami,t_cost_nmi,auto_ownership_cost_ami,auto_ownership_cost_80ami,auto_ownership_cost_nmi,vmt_cost_ami,vmt_cost_80ami,vmt_cost_nmi,transit_cost_ami,transit_cost_80ami,transit_cost_nmi,transit_trips_ami,transit_trips_80ami,transit_trips_nmi,compact_ndx,emp_ovrll_ndx,res_density,gross_hh_density,hh_gravity,frac_sfd,emp_gravity,emp_ndx,block_size,intersection_density,avg_block_perimeter_meters,h_cost,median_smoc,median_gross_rent,pct_owner_occupied_hu,pct_renter_occupied_hu,state,GEOID,year,name,parent-location,population_y,poverty-rate,renter-occupied-households,pct-renter-occupied,median-gross-rent,median-household-income,median-property-value,rent-burden,pct-white,pct-af-am,pct-hispanic,pct-am-ind,pct-asian,pct-nh-pi,pct-multiple,pct-other,eviction-filings,evictions,eviction-rate,eviction-filing-rate,low-flag,imputed,subbed
0,0,4.013061e+10,"Phoenix-Mesa-Scottsdale, AZ",1.0,603.0,161.0,161.70,42.0,50.0,42.0,19.0,24.0,19.0,23.0,26.0,23.0,8.63,8.60,1.67,2.0,2.0,20079.0,17748.0,19925.0,3.0,4.0,3.0,12338.0,11151.0,12322.0,9106.0,8292.0,9115.0,3220.0,2846.0,3195.0,12.0,13.0,12.0,13.0,14.0,13.0,7.5,5.3,2.20,1.00,25032.0,55.0,14458.0,88.0,10.0,109.0,828,860.0,1039.0,742.0,40.0,60.0,az,4.013061e+10,2016,609.04.1,"Maricopa County, Arizona",603.0,19.27,123.0,60.25,742.0,33625.0,57300.0,18.1,34.83,0.83,60.36,0.00,0.00,0.00,0.00,3.98,NaN,NaN,NaN,NaN,0,0,0
1,1,4.013110e+10,"Phoenix-Mesa-Scottsdale, AZ",1.0,2283.0,550.0,208.83,42.0,50.0,42.0,19.0,24.0,19.0,23.0,26.0,23.0,8.01,21.10,1.72,2.0,2.0,18699.0,16734.0,18581.0,6.0,7.0,6.0,12399.0,10982.0,12316.0,9379.0,8276.0,9315.0,2999.0,2683.0,2980.0,22.0,23.0,22.0,24.0,26.0,24.0,7.0,5.0,9.84,2.63,34717.0,70.0,36272.0,92.0,8.0,187.0,727,861.0,918.0,799.0,52.0,48.0,az,4.013110e+10,2016,1099.3,"Maricopa County, Arizona",2283.0,43.28,271.0,48.18,799.0,21959.0,55500.0,33.8,5.48,8.54,85.98,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,0,0,0
2,2,4.013061e+10,"Phoenix-Mesa-Scottsdale, AZ",1.0,824.0,220.0,117.03,52.0,62.0,52.0,27.0,33.0,27.0,25.0,29.0,25.0,11.31,21.25,1.86,2.0,2.0,22311.0,19998.0,22158.0,2.0,2.0,2.0,13682.0,12387.0,13645.0,10098.0,9174.0,10086.0,3578.0,3207.0,3553.0,6.0,6.0,6.0,7.0,7.0,7.0,5.7,2.2,3.44,1.88,18034.0,88.0,8027.0,87.0,9.0,283.0,1114,1191.0,1432.0,880.0,56.0,44.0,az,4.013061e+10,2016,610.32.4,"Maricopa County, Arizona",824.0,0.00,81.0,43.64,880.0,81250.0,157900.0,16.7,54.73,5.58,29.25,4.73,0.00,0.00,5.70,0.00,NaN,NaN,NaN,NaN,0,0,0
3,3,4.013217e+10,"Phoenix-Mesa-Scottsdale, AZ",1.0,201.0,130.0,72.82,58.0,68.0,57.0,34.0,42.0,34.0,24.0,26.0,24.0,5.01,8.94,1.76,2.0,2.0,20321.0,17604.0,20073.0,3.0,3.0,3.0,12837.0,11259.0,12708.0,9567.0,8425.0,9477.0,3259.0,2823.0,3219.0,12.0,11.0,12.0,13.0,12.0,13.0,5.9,7.1,28.33,1.79,24598.0,54.0,39081.0,91.0,5.0,220.0,707,1514.0,1514.0,NaN,88.0,12.0,az,4.013217e+10,2016,2169.01.3,"Maricopa County, Arizona",201.0,12.50,55.0,11.54,NaN,68194.0,283000.0,NaN,95.52,0.00,4.48,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,0,0,0
4,4,4.021001e+10,"Phoenix-Mesa-Scottsdale, AZ",1.0,2653.0,827.0,30248.83,55.0,65.0,54.0,26.0,32.0,26.0,29.0,33.0,29.0,10.62,0.29,2.10,2.0,2.0,25504.0,22972.0,25294.0,0.0,0.0,0.0,15530.0,14294.0,15487.0,11440.0,10610.0,11431.0,4090.0,3684.0,4056.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.2,0.79,0.03,1793.0,74.0,1740.0,83.0,121.0,16.0,2336,1149.0,1293.0,907.0,63.0,37.0,az,4.021001e+10,2016,12.1,"Pinal County, Arizona",2653.0,8.09,226.0,37.24,907.0,54922.0,126300.0,31.9,47.57,0.94,39.39,6.71,1.06,0.57,3.77,0.00,NaN,NaN,NaN,NaN,0,0,0


In [7]:
# build dataset restricting data to eviction-rate < 100

analysis_filtered_df = analysis_df.loc[(analysis_df['eviction-rate']<=100) & (analysis_df['population_x']>0)].copy()

## Create list of CBSAs

In [8]:
cbsa_list = analysis_filtered_df['cbsa'].unique().tolist()

In [9]:
cbsa_list

['Los Angeles-Long Beach-Anaheim, CA',
 'San Francisco-Oakland-Hayward, CA',
 'Riverside-San Bernardino-Ontario, CA',
 'San Diego-Carlsbad, CA',
 'Washington-Arlington-Alexandria, DC-VA-MD-WV',
 'Philadelphia-Camden-Wilmington, PA-NJ-DE-MD',
 'Miami-Fort Lauderdale-West Palm Beach, FL',
 'Tampa-St. Petersburg-Clearwater, FL',
 'Atlanta-Sandy Springs-Roswell, GA',
 'Chicago-Naperville-Elgin, IL-IN-WI',
 'St. Louis, MO-IL',
 'Boston-Cambridge-Newton, MA-NH',
 'Detroit-Warren-Dearborn, MI',
 'Minneapolis-St. Paul-Bloomington, MN-WI',
 'New York-Newark-Jersey City, NY-NJ-PA',
 'Dallas-Fort Worth-Arlington, TX',
 'Houston-The Woodlands-Sugar Land, TX',
 'Seattle-Tacoma-Bellevue, WA']

## Clean column names

In [10]:
columns = analysis_filtered_df.columns

In [11]:
updated_columns = []

for name in columns:
    new_name = name.replace('-', '_')
    updated_columns.append(new_name)

In [12]:
updated_columns

['Unnamed: 0',
 'blkgrp',
 'cbsa',
 'blkgrps',
 'population_x',
 'households',
 'land_acres',
 'ht_ami',
 'ht_80ami',
 'ht_nmi',
 'h_ami',
 'h_80ami',
 'h_nmi',
 't_ami',
 't_80ami',
 't_nmi',
 'co2_per_hh_local',
 'co2_per_acre_local',
 'autos_per_hh_ami',
 'autos_per_hh_80ami',
 'autos_per_hh_nmi',
 'vmt_per_hh_ami',
 'vmt_per_hh_80ami',
 'vmt_per_hh_nmi',
 'pct_transit_commuters_ami',
 'pct_transit_commuters_80ami',
 'pct_transit_commuters_nmi',
 't_cost_ami',
 't_cost_80ami',
 't_cost_nmi',
 'auto_ownership_cost_ami',
 'auto_ownership_cost_80ami',
 'auto_ownership_cost_nmi',
 'vmt_cost_ami',
 'vmt_cost_80ami',
 'vmt_cost_nmi',
 'transit_cost_ami',
 'transit_cost_80ami',
 'transit_cost_nmi',
 'transit_trips_ami',
 'transit_trips_80ami',
 'transit_trips_nmi',
 'compact_ndx',
 'emp_ovrll_ndx',
 'res_density',
 'gross_hh_density',
 'hh_gravity',
 'frac_sfd',
 'emp_gravity',
 'emp_ndx',
 'block_size',
 'intersection_density',
 'avg_block_perimeter_meters',
 'h_cost',
 'median_smoc',
 'm

In [13]:
analysis_filtered_df.columns = updated_columns

# Analysis

**Columns of Interest:** 
1. Income (median_household_income)
2. Percent Renter Occupied Housing Units (pct_renter_occupied_hu)
3. Percent African American (pct_af_am)
4. Percent Single Family Detached Households (frac_sfd)
5. Transit Ridership % of Workers for the Regional Typical Household (pct_transit_commuters_ami)
6. Job Access Score (emp_ovrll_ndx)

In [14]:
summary_columns = ['regression_name', 'regression_note', 'cbsa', 'num_obs', 'r_squared', 'key', 'coeffs', 'std_errs', 't_values', 'p_values']

In [16]:
# iterate through cbsa_lists and generate models

# predictions_ht = {}
summary_results_six_var = {}

# iterate through each CBSA and output the results
for cbsa in cbsa_list:
    print('cbsa: '+cbsa)
    
    # create DataFrame for analysis within for loop (limited to one cbsa)
    iterated_df = analysis_filtered_df.loc[analysis_filtered_df['cbsa'] == cbsa].copy()
    
    
   # final data clean
    iterated_df.fillna(value=0)

    # create model & prediction
    cbsa_model = smf.ols('eviction_rate ~ median_household_income + pct_renter_occupied_hu + pct_af_am + frac_sfd + pct_transit_commuters_ami + emp_ovrll_ndx', data = iterated_df).fit()
    
    #### Output Variables
    regression_name = '20200503 selected variables'
    regression_note = 'six variables identified by the team - no corhorting'
    
    num_obs = cbsa_model.nobs
    r_squared = cbsa_model.rsquared
    

    coeffs = {}
    std_errs = {}
    t_values = {}
    p_values = {}    

    coeffs = dict(cbsa_model.params)
    std_errs = dict(cbsa_model.bse)
    t_values = dict(cbsa_model.tvalues)
    p_values = dict(cbsa_model.pvalues)
    
    output_values = {}
    
    print(coeffs)
    
    for key in coeffs.keys():
        values_list = [regression_name, regression_note, cbsa, num_obs, r_squared, key, coeffs[key], std_errs[key], t_values[key], p_values[key]]
        output_values[key] = values_list
        
    output_df = pd.DataFrame.from_dict(output_values, orient='index',columns=summary_columns)
    
    summary_results_six_var[cbsa] = output_df

#     # store model & preduction
#     predictions_ht[cbsa] = [cbsa_model,cbsa_predictions]
    
    # Print out the statistics and plot
    print(cbsa_model.summary())
    print('### pvalues:')
    print(cbsa_model.pvalues)
    print('##############################################')

cbsa: Los Angeles-Long Beach-Anaheim, CA
{'Intercept': 1.2450285828492156, 'median_household_income': -1.8629543254349453e-06, 'pct_renter_occupied_hu': -0.0024312453175499545, 'pct_af_am': 0.010254212018228862, 'frac_sfd': -0.0009540519455911963, 'pct_transit_commuters_ami': -0.00807669808318648, 'emp_ovrll_ndx': -0.053931768838737695}
                            OLS Regression Results                            
Dep. Variable:          eviction_rate   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     44.62
Date:                Sun, 03 May 2020   Prob (F-statistic):           5.52e-54
Time:                        13:08:35   Log-Likelihood:                -11999.
No. Observations:                8033   AIC:                         2.401e+04
Df Residuals:                    8026   BIC:                         2.406e+04
Df Model:                    

In [17]:
summary_results_six_var_df = pd.concat(summary_results_six_var.values(), ignore_index=True)

In [18]:
summary_results_six_var_df

,regression_name,regression_note,cbsa,num_obs,r_squared,key,coeffs,std_errs,t_values,p_values
0,20200503 selected variables,six variables identified by the team - no corh...,"Los Angeles-Long Beach-Anaheim, CA",8033.0,0.032281,Intercept,1.245029,9.081236e-02,13.709902,2.648086e-42
1,20200503 selected variables,six variables identified by the team - no corh...,"Los Angeles-Long Beach-Anaheim, CA",8033.0,0.032281,median_household_income,-0.000002,4.303564e-07,-4.328864,1.516997e-05
2,20200503 selected variables,six variables identified by the team - no corh...,"Los Angeles-Long Beach-Anaheim, CA",8033.0,0.032281,pct_renter_occupied_hu,-0.002431,8.637195e-04,-2.814855,4.891809e-03
3,20200503 selected variables,six variables identified by the team - no corh...,"Los Angeles-Long Beach-Anaheim, CA",8033.0,0.032281,pct_af_am,0.010254,9.501066e-04,10.792696,5.705028e-27
4,20200503 selected variables,six variables identified by the team - no corh...,"Los Angeles-Long Beach-Anaheim, CA",8033.0,0.032281,frac_sfd,-0.000954,6.360658e-04,-1.499927,1.336728e-01
...,...,...,...,...,...,...,...,...,...,...
121,20200503 selected variables,six variables identified by the team - no corh...,"Seattle-Tacoma-Bellevue, WA",2440.0,0.077454,pct_renter_occupied_hu,-0.000199,1.754630e-03,-0.113568,9.095893e-01
122,20200503 selected variables,six variables identified by the team - no corh...,"Seattle-Tacoma-Bellevue, WA",2440.0,0.077454,pct_af_am,0.007191,2.901936e-03,2.477840,1.328546e-02
123,20200503 selected variables,six variables identified by the team - no corh...,"Seattle-Tacoma-Bellevue, WA",2440.0,0.077454,frac_sfd,-0.004345,1.373049e-03,-3.164839,1.570857e-03
124,20200503 selected variables,six variables identified by the team - no corh...,"Seattle-Tacoma-Bellevue, WA",2440.0,0.077454,pct_transit_commuters_ami,-0.044432,5.872744e-03,-7.565877,5.429128e-14


In [20]:
if export_files == True:
    summary_results_six_var_df.to_csv(export_directory+'20200503 Six Variable Linear Regression.csv')